<a href="https://colab.research.google.com/github/francji1/01ZLMA/blob/main/R/01ZMLA_ex06_cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyse data of car accidents in Sweeden.

In [ ]:
library(tidyverse)
library(MASS)
library(knitr)

install.packages("GGally")
library(GGally)

#install.packages("car")
#library(car)

In [ ]:
sweden2 <- "https://raw.githubusercontent.com/francji1/01ZLMA/main/data/sweden.csv"
cars    <- read.table(sweden2, header = T, sep = ",")
summary(cars)
head(cars)



Dataset contains the number of deaths, he number of registered cars, the annual volume of sold fuels, the number of registered vehicles and the year. Our goal will be to create the best possible model for death rates. Since these are the number of events per unit time, we use the Poisson distribution with the *canonical link function*  $g (\mu) = log(\mu) $.

Because the observed period is relatively long and there have been significant changes in transport, we will only consider data from 1975.

In [ ]:
par(mfrow=c(1,2))
plot(cars$Deaths~cars$Year, col="red", lwd=2)
cars <- cars[cars$Year>1974,]
plot(cars$Deaths~cars$Year, col="red", lwd=2)

In [ ]:
attach(cars)

In [ ]:
ggpairs(cars)

# Tasks 

* Find the best posssible model. Try add all varariables up to the second order interactions. You can use `step()` function based on `AIC`.
* Analyse residuals and check systematic components. 
* If incorrect linear predictor appears, try to transform corresponding variable. Hint: `Fuel_transformed <- log(abs(Fuel-mean(Fuel)))`
* Run post hoc analysis of your final model again
* Find influence observations and decide how to cope with them (if any)
* Hide last 5 observations, train the model using remaining and try to predict response for the 5 hidden latest observations. Plot predictiions together with true observations.


## Solution by T.H.

In [ ]:
attach(cars)

##  Model s jedinou proměnnou Year

In [ ]:
mdl_0 <- glm(formula=Deaths~Year,family=poisson(link = "log"),data=cars)
#summary(mdl_0)
printCoefmat(coef(summary(mdl_0)))
fit_0 <- predict(mdl_0, type = "response")

##  Model se všemi proměnnými bez interakcí

In [ ]:
mdl_1 <- glm(formula=Deaths~Year+Cars+Fuel,family=poisson(link = "log"),data=cars)
#summary(mdl_1)
printCoefmat(coef(summary(mdl_1)))
fit_1 <- mdl_1$fitted.values

In [ ]:
plot(Year, Deaths, col="red", cex=1, lwd=2)
lines(Year, fit_0, col="blue", type = "l", lwd=2)
lines(Year, fit_1, col="brown", type = "l", lwd=2)
legend("topright",  legend = c("Data", "Model 0", "Model 1"), 
   col = c("red","blue","brown"), bty="n", lwd = 2, pch=c(1,NA,NA), cex=0.9, lty=c(0,1,1))

In [ ]:
#library(ggplot2)
#install.packages("gridExtra")
#library(gridExtra)


p <- ggplot(cars %>%  mutate(model_0 = mdl_0$fitted.values,
                 model_1 = mdl_1$fitted.values) %>%
            pivot_longer(cols = starts_with("model"),
                         names_to = "model",
                         values_to = "fit"),
            aes(x = Year, y = Deaths, colour = model)) + 
      geom_point(color = "black") +
      geom_line(aes(y = fit), size = 1)
p

In [ ]:
AIC_0 <- mdl_0$aic
AIC_1 <- mdl_1$aic
c(AIC_0, AIC_1)

## Model s interakcemi druhého řádu

In [ ]:
mdl_2 <- glm(formula=Deaths~(.)^2,family=poisson(link = "log"),data=cars)
#mdl_2 <- glm(formula=Deaths~(Year+Cars+Fuel)^2,family=poisson(link = "log"))

AIC_2 <- mdl_2$aic
AIC_2
printCoefmat(coef(summary(mdl_2)))

Vidíme, že hodnota AIC se opět snížila, i když interakce se v modelu zdají nevýznamné. Odstraníme tu nejméně významnou dle p-hodnoty, tedy `Cars:Fuel`.  Další možnost, jak se rozhodnout, kterou proměnnou z modelu vypustit nabízí použití funkce `drop1()`,

In [ ]:
drop1(mdl_2, test="Chisq")

která kromě p-hodnot vrací i deviaci a AIC statistiku pro model po vynechání dané proměnné. Vidíme i zde, že vynechání interakce `Cars:Fuel` má za následek nejmenší nárůst deviace a největší pokles AIC. Vynecháme ji tedy

In [ ]:
mdl_3 <- update(mdl_2, ~. - Cars:Fuel)  
# mdl_3 <- glm(formula=Deaths~(.)^2-Cars:Fuel,family=poisson(link = "log"),data=cars)
printCoefmat(coef(summary(mdl_3)))
AIC_3 <- mdl_3$aic
AIC_3

Hodnota AIC se opět snížila, vynecháme další nevýznamnou interakci `Year:Cars`

In [ ]:
mdl_4 <- update(mdl_3, ~. - Year:Cars)
printCoefmat(coef(summary(mdl_4)))
AIC_4 <- mdl_4$aic; AIC_4

Opět došlo ke snížení AIC a všechny proměnné a interakce už jsou v modelu významné.

Porovnejme ještě modely mezi sebou. Nejdříve Model 4 a Model 3

In [ ]:
anova(mdl_4,mdl_3, test="Chisq")

Vidíme, že mezi nimi není významný rozdíl, jako lepší tedy bereme Model 4, který má méně parametrů (tomu odpovídá i hodnota AIC). Ještě porovnáme Model 1 bez interakcí s Modelem 4 s jednou interakcí

In [ ]:
anova(mdl_1,mdl_4, test="Chisq")

Tenokráte je rozdíl významný, jako prozatím finální model tedy budeme uvažovat Model 4.

In [ ]:
fit_4 <- predict(mdl_4, type = "response")
plot(Year, Deaths, col="red", lwd=2)
lines(Year, fit_4, col="blue", type = "l", lwd=2)

In [ ]:
p <- ggplot(cars %>%  mutate(model_0 = mdl_0$fitted.values,
                 model_1 = mdl_1$fitted.values,
                 model_4 = mdl_4$fitted.values) %>%
            pivot_longer(cols = starts_with("model"),
                         names_to = "model",
                         values_to = "fit"),
            aes(x = Year, y = Deaths, colour = model)) + 
      geom_point(color = "black") +
      geom_line(aes(y = fit), size = 1)
p

K výběru vhodných proměnných lze použít i automatickou proceduru zpětného výběru pomocí funkce `step()`,

In [ ]:
mdl_4b <- step(mdl_2, direction="backward",k=2,trace=1)
summary(mdl_4b)

která v tomto případě vrátí stejný model.

## Analýza reziduí a influenčních pozorování

In [ ]:
r.ds <- rstandard(mdl_4, type = "deviance")  # standardizovaná deviační rezidua

In [ ]:
par(mfrow=c(2,2))
# obrázky standardizovaných deviačních reziduí
scatter.smooth( r.ds ~ fit_4, las=1,
                xlab="Fitted values", ylab="Standardized residuals")
scatter.smooth( r.ds ~ Year, las=1,
                xlab="Year", ylab="Standardized residuals")
scatter.smooth( r.ds ~ Cars, las=1,
                xlab="Cars", ylab="Standardized residuals")
scatter.smooth( r.ds ~ Fuel, las=1,
                xlab="Fuel", ylab="Standardized residuals")

Obrázky reziduí vypadají poměrně dobře až na obrázek vpravo dole pro proměnnou `Fuel`. To by mohlo značit, že není v modelu obsažena správně, např. může chybět nějaká její mocnina nebo jiná transformace. Malé a velké hodnoty `FUEL` mají za následek záporná rezidua, proměnnou jsem tedy vystředoval a po chvíli zkoušení našel následující transformaci

In [ ]:
Fuel.t <- log(abs(Fuel-mean(Fuel)))
cars <- cbind(cars, Fuel.t)

kterou přidáme do modelu

In [ ]:
mdl_5 <- update(mdl_4, ~. +Fuel.t)
summary(mdl_5)

Všecny parametry jsou v modelu významné a navíc došlo kdalšímu poklesu hodnoty AIC. Porovnejme ještě modely 4 a 5 pomocí LRT testu

In [ ]:
anova(mdl_4,mdl_5, test="Chisq")

Rozdíl je významný, lepší je tedy Model 5. Nelze tvrdit, že je to nejlepší možný model, jistě by šly najít i jiné transformace, případně transformace, které by se aplikovali na proměnnou `FUEL` už na začátku modelování. To ale nechám na případné domácí cvičení. 

Podívejme se rezidua Modelu 5

In [ ]:
fit_5 <- fitted(mdl_5)
r.ds <-  rstandard(mdl_5)

In [ ]:
par(mfrow=c(2,3))
# obrazky standardizovanych deviacnich rezidui
scatter.smooth( r.ds ~ fit_5, las=1,
                xlab="Fitted values", ylab="Standardized residuals")
scatter.smooth( r.ds ~ Year, las=1,
                xlab="Year", ylab="Standardized residuals")
scatter.smooth( r.ds ~ Cars, las=1,
                xlab="Cars", ylab="Standardized residuals")
scatter.smooth( r.ds ~ Fuel, las=1,
                xlab="Fuel", ylab="Standardized residuals")
scatter.smooth( r.ds ~ Fuel.t, las=1,
                xlab="Fuel.t", ylab="Standardized residuals")


Rezidua teď vypadají mnohem rovnoměrněji rozdělena.Podívejme se ješte na vhodnost použité spojovací funkce a normalitu reziduí

In [ ]:
par(mfrow=c(1,2))
eta <- mdl_5$linear.predictor
z <- resid(mdl_5, type="working") + eta
plot( z ~ eta, las=1,
      xlab="Linear predictor, eta", ylab="Working responses, z")
abline(0, 1, col="grey")
qqnorm(r.ds)
qqline(r.ds)

Co se týče spojovací funkce, zdá být vše v pořádku. Rezidua vykazují jistou odchylky od normality, ale připomeňme, že v poissonovské regresi by měla být rezidua "přibližně" normálně rozdělena.

## Analýza influenčních pozorování 

In [ ]:
par(mfrow=c(1,2))
npar <- 6
n<- 36
cd <- cooks.distance(mdl_5)
plot(cd,col="red", lwd=2)
abline(8/(n-2*npar),0)
lev<-hatvalues(mdl_5)
plot(lev,col="red", lwd=2)
abline(2*npar/(n),0)

Pozorování 36 bylo označeno jako influenční. Pokud použijeme i funkci pro počítání influenčních měr

In [ ]:
im <- influence.measures(mdl_3); names(im)
colSums( im$is.inf )
which(im$is.inf[,9]==TRUE)

dostaneme, že jedno pozorování bylo detekováno jako influenční pomocí Cookovy vzdálenosti a jedná se o pozorování 36. Naopak pomocí potenciálů `hat` žádné podezřelé pozorování označeno nebylo.



Dojdeme-li k závěru, že pozorování 36 je "špatné", můžeme ho odstranit z modelu. Potom 

In [ ]:
mdl_5.out <- update(mdl_5, subset = -36)
coef(mdl_5.out)
coef(mdl_5)

Vidíme, že došlo k mírné změně koeficientů. Rozdíl v odhadech.

In [ ]:
fit_5.out <- predict(mdl_5.out, newdata = cars, type = "response")
plot(Year, Deaths, col="red", lwd=2)
lines(Year, fit_5, col="blue", type = "l", lwd=3)
lines(Year, fit_5.out, col="green", type = "l", lty=6, lwd=2)
legend("topright", legend = c("Data", "mdl_5", "mdl_5.out"),
    col = c("red","blue","green"), bty="n", lwd = 2, pch=c(1,NA,NA), lty=c(0,1,1))

se však tato změna projeví jen nepatrně u posledního pozorování.





Last task: Hide last 5 observations, train the model using remaining and try to predict response for the 5 hidden latest observations. Plot predictiions together with true observations.

In [ ]:
library(tidyverse)

In [ ]:
sweden2 <- "https://raw.githubusercontent.com/francji1/01ZLMA/main/data/sweden.csv"
cars <- read.table(sweden2, header = T, sep = ",") %>% 
         mutate(Fuel_t = log(abs(Fuel-mean(Fuel)))) %>%
         filter(Year > 1974) #%>% 
         #dplyr::select(-Fuel) 

head(cars)
train <- cars[1:(dim(cars)[1]-5),]
test  <- cars[-c(1:(dim(cars)[1]-5)),]
dim(train)
dim(test)

In [ ]:
model_cars0 <- glm(Deaths~Year+Cars,family=poisson(link = "log"),data=train)
summary(model_cars0)

model_cars1 <- glm(formula=Deaths~(.)^2,family=poisson(link = "log"),data=train) %>%
              step(direction="backward",k=2,trace=0)
summary(model_cars1)

model_cars2 <- glm(Deaths~Year+Cars+ Fuel + Fuel_t + Year:Fuel,family=poisson(link = "log"),data=train)
summary(model_cars2)



In [ ]:
predict(model_cars, newdata = test, type = c("response"), se.fit = TRUE)$fit

In [ ]:
cars2 <-cars %>%  mutate(model_0 = c(model_cars0$fitted.values,predict(model_cars0, newdata = test, type = c("response"), se.fit = TRUE)$fit),
                 model_1 = c(model_cars1$fitted.values,predict(model_cars1, newdata = test, type = c("response"), se.fit = TRUE)$fit),
                 model_2 = c(model_cars2$fitted.values,predict(model_cars2, newdata = test, type = c("response"), se.fit = TRUE)$fit),
                 type = c(rep("train",31),rep("test",5)),
                 type = as.factor(type)) %>%
            pivot_longer(cols = starts_with("model"),
                         names_to = "model",
                         values_to = "fit")

p <- ggplot(cars2, aes(x = Year, y = fit, color = model)) + 
      geom_point(aes(y = Deaths)) +
      geom_line(aes(linetype=type, color=model), size = 1)
p